# Config Static Route

https://blog.csdn.net/chwzmx123/article/details/79425551

<img src="Config Static Route.png" alt="drawing" width="500"/>

# router: deal with IP addresses

https://www.lifewire.com/how-routers-work-816456

### a router is a Layer 3 network gateway device, meaning that it connects two or more networks and that the router operates at the network layer of the OSI model. 

### Routers contain a processor (CPU), several kinds of digital memory, and input-output (I/O) interfaces. They function as special-purpose computers, one that does not require a keyboard or display.

### By maintaining configuration information in a part of memory called the routing table, routers also can filter both incoming or outgoing traffic based on the addresses of senders and receivers.

### Home networks use Internet Protocol (IP) routers to connect computers to each other and to the Internet. Early generations of home routers supported wired networking with Ethernet cables while newer wireless routers supported Wi-Fi together with Ethernet. The term broadband router applies to any home wired or wireless router being used for sharing a broadband Internet connection.

# switch: deal with MAC addresses

https://www.lifewire.com/definition-of-network-switch-817588

### A network switch is a small hardware device that centralizes communications among multiple connected devices in one local area network (LAN).

### Stand-alone Ethernet switch devices were commonly used on home networks many years before home broadband routers became popular. Modern home routers integrate Ethernet switches directly into the unit as one of their core functions.

### High-performance network switches are still widely used in corporate networks and data centers. Network switches are sometimes referred to as switching hubs, bridging hubs or MAC bridges.

### Traditional network switches operate at Layer 2 Data Link Layer of the OSI model.

### 与桥接器一样，交换机按每一个包中的MAC地址相对简单地决策信息转发。而这种转发决策一般不考虑包中隐藏的更深的其他信息。与桥接器不同的是交换机转发延迟很小，操作接近单个局域网性能，远远超过了普通桥接互联网网络之间的转发性能。

### 交换机内部的CPU会在每个端口成功连接时，通过将MAC地址和端口对应，形成一张MAC表。在今后的通讯中，发往该MAC地址的数据包将仅送往其对应的端口，而不是所有的端口。

### 交换机拥有一条很高带宽的背部总线和内部交换矩阵。交换机的所有的端口都挂接在这条背部总线上，控制电路收到数据包以后，处理端口会查找内存中的地址对照表以确定目的MAC（网卡的硬件地址）的NIC（网卡）挂接在哪个端口上，通过内部交换矩阵迅速将数据包传送到目的端口，目的MAC若不存在，广播到所有的端口，接收端口回应后交换机会“学习”新的MAC地址，并把它添加入内部MAC地址表中。

## find changed files (想知道孙煜学长改了哪些代码)

https://www.2cto.com/os/201211/170050.html

find ./ -mtime -200 -name "*.h"

<img src="find changed files.png" alt="drawing" width="600"/>

# ECMP

https://blog.csdn.net/Mary19920410/article/details/73499630

### Equal-Cost Multi-path Routing，等价多路径。即存在多条到达同一个目的地址的相同开销的路径。当设备支持等价路由时，发往该目的 IP 或者目的网段的三层转发流量就可以通过不同的路径分担，实现网络的负载均衡，并在其中某些路径出现故障时，由其它路径代替完成转发处理，实现路由冗余备份功能。

### Equal-Cost Multi-Path (ECMP) is a forwarding mechanism for routing packets along multiple paths of equal cost with the goal to achieve almost equally distributed link load sharing.

### 如果使用传统的路由技术，发往该目的地址的数据包只能利用其中的一条链路，其它链路处于备份状态或无效状态，并且在动态路由环境下相互的切换需要一定的时间，而等价多路径路由协议可以在该网络环境下同时使用多条链路，不仅增加了传输带宽，并且可以无时延无丢包地备份失效链路的数据传输。

### 分配包的基本方式

### 1）基于数据流的负载分担

### 目的地址和源地址相同的报文属于一个数据流。基于数据流的负载分担的方式就是，假定有10个数据流，有2条路径可选择，一边各走5个。

### 2）基于数据报文的负载分担

### 假定有10个数据报文，有2个路径可选择，一边各走5个。

# CONGA

<img src="Single rooted tree.png" alt="drawing" width="500"/>

### Single rooted tree structure is being replaced or has been replaced by Multi rooted tree structure (e.g., Leaf-Spine as shown in the pic below).

### Benefits: you can scale the bisection bandwidth of the network as much as you want by just creating more paths through the topology.

<img src="Multi-rooted tree.png" alt="drawing" width="500"/>

### For example, if I want more bandwidth, I can add a couple of spine switches and I have a network with higher aggregate capacity.

<img src="add more spines.png" alt="drawing" width="500"/>

### 但想用好这个结构，需要solve a load balancing problem: distributing traffic across different paths.

### 目前的ECMP：every switch will pick the next hop or pick the path for a packet by taking a hash of the packets' headers (which is a 5-tuple: source IP, destination IP, source port, destination port, protocol).

### What's good about it is that it does this while preserving packet order. 比如：TCP could get confused by packet reordering and think it's a sign of congestion in the network.

### ECMP的问题：1）hash collisions；2）it simply makes a purely local and stateless decision. The decision has nothing to do with the actual state of congestion in the network. ECMP can perform really poorly if the topology isn't perfectly symmetric. It is oblivious to congestion (作者做了一个比喻：San Francisco to San Jose两条路101, 280，ECMP通过抛硬币决定走哪条。当101路堵了，ECMP有50%的机率依然选择那条堵的。)

## CONGA：congestion-aware

### make intelligent decisions based on how much congestion there is on each path.

### It's not enough to measure congestion on just the first hop (local congestion aware scheme at the switch is not enough). You need to be aware of the end-to-end path congestion.

### Every leaf switch (or top-of-rack switch) will track congestion metrics on a per path basis from themselves to every other leaf switch. And they will simply always try to send traffic on only the least congested path.

### We do this in CONGA by implementing these extremely fast feedback loops between the leaf switches directly in the hardware data plane.

<img src="fast feedback loops-1.png" alt="drawing" width="500"/>
<img src="fast feedback loops-2.png" alt="drawing" width="500"/>

### CONGA operates over a standard DC overlay (VXLAN). Host 1发包至Host 9的过程：先到达Leaf Switch 0，随即L0 encapsulate the packet with an overlay header which is VXLAN encap ("L0 -> L2"). L2 removes the overlay header and send the packet onward to H9.
<img src="Congestion-To-Leaf.png" alt="drawing" width="500"/>
### We are trying to track this per path congestion metric between the leaf switches. Every leaf switch maintain a table: Congestion-To-Leaf Table. The goal of the feedback mechanism is to populate metrics in this table. The metrics are 3-bit values.

### {Rate Measurement Module} measures link utilization (e.g., L0 -> S0, S0 -> L1, ...). The metric for a path is the maximum of congestion across all of its links. CE: Congestion Experience

### Feedback process: piggyback it on packets that are going in the reverse direction. So the switch L2 here will temporarily store the information and then anytime there is a packet going in the reverse direction it will pick the metric from L2 and carry it back to L0 and populate the "congestion to leaf" table at L0 which is the one that we set out to populate.
<img src="leaf-to-leaf-feedback.png" alt="drawing" width="500"/>

### We want to make sure that there would be no reordering for TCP traffic, so we don't actually do this on a packet by packet basis to avoid packet reordering, we make decisions on a flowlet basis. 

### A flowlet is a burst of packets that are separated from other bursts by a large enough gap that you can send one burst on one path and another burst on a different path without causing reordering. 

### Per Flow. 这样控制粒度不够细化，flow的差异非常大；而Per Packet呢，粒度够细，但是要面对失序问题（reordering），跟TCP相关了。所以，Per Flowlet最好，不需要修改TCP，而且粒度也够细。
<img src="Flowlet-gap.png" alt="drawing" width="500"/>

https://blog.csdn.net/sinat_21645561/article/details/48637209#conga
<img src="Tree structure of LB design.png" alt="drawing" width="800"/>

### In packet switching networks, traffic flow, packet flow or network flow is a sequence of packets from a source computer to a destination, which may be another host, a multicast group, or a broadcast domain. RFC 2722 defines traffic flow as "an artificial logical equivalent to a call or connection."[1] RFC 3697 defines traffic flow as "a sequence of packets sent from a particular source to a particular unicast, anycast, or multicast destination that the source desires to label as a flow. A flow could consist of all packets in a specific transport connection or a media stream. However, a flow is not necessarily 1:1 mapped to a transport connection."[2] Flow is also defined in RFC 3917 as "a set of IP packets passing an observation point in the network during a certain time interval."[3]


### What is Cisco ACI? This is part one of SDxCentral’s series to explain the Cisco ACI (Application Centric Infrastructure) framework and how it’s the Cisco SDN Solution.
https://www.sdxcentral.com/data-center/definitions/what-is-cisco-aci/

# Reinforcement Learning

### First, agent’s actions are determined by a stochastic policy π(s). Stochastic policy means that it does not output a single action, but a distribution of probabilities over actions, which sum to 1.0. We’ll also use a notation π(a‖s) which means the probability of taking action a in state s.

### 先走出 N_STEP_RETURN 步，之后调用get_sample函数，以计算 (s0,a0,R,sn)并送给train函数以优化神经网络参数，将memory中的 (s0,a0,r0,s1) 删除，再继续走下一步，计算出 (s1,a0,R',s_n+1)，将memory中的 (s1,a1,r1,s2) 删除，直到when our agent encounters a terminal state，we have to deplete the whole buffer：依次将memory中的head删除，删除前计算 (s_i, a_i, r_i, s_end)，直至 (s_end-1, a_end-1, r_end-1, s-end)，将memory中的信息全部利用完，删干净。

### self.memory.append( (s, a_cats, r, s_) )
### if len(self.memory) >= N_STEP_RETURN:
###            s, a, r, s_ = get_sample(self.memory, N_STEP_RETURN)
###            brain.train_push(s, a, r, s_)
###            self.memory.pop(0)

### landing state after n steps s_ and a terminal mask with values 1. or 0., indicating whether the s_ is None or not

### an off-policy learning algorithm. The basic idea is to choose actions according to a stochastic behaviour policy (to ensure adequate exploration), but to learn about a deterministic target policy (exploiting the ef- ficiency of the deterministic policy gradient)